In [7]:
import random
import this

loc_A, loc_B = (0, 0), (1, 0)

class Thing:
    """This represents any physical object that can appear in an Environment.
    You subclass Thing to get the things you want. Each thing can have a
    ._name_  slot (used for output only)."""

    def _repr_(self):
        return '<{}>'.format(getattr(self, '_name', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Display the agent's internal state. Subclasses should override."""
        print("I don't know how to show_state.")

    def display(self, canvas, x, y, width, height):
        """Display an image of this Thing on the canvas."""
        # Do we need this?
        pass



class Agent(Thing):
    """An Agent is a subclass of Thing with one required slot,
    .program, which should hold a function that takes one argument, the
    percept, and returns an action. (What counts as a percept or action
    will depend on the specific environment in which the agent exists.)
    Note that 'program' is a slot, not a method. If it were a method,
    then the program could 'cheat' and look at aspects of the agent.
    It's not supposed to do that: the program can only look at the
    percepts. An agent program that needs a model of the world (and of
    the agent itself) will have to build and maintain its own model.
    There is an optional slot, .performance, which is a number giving
    the performance measure of the agent in its environment."""
    def _init_(self, program):
        self.program = program

def TraceAgent(agent):
    """Wrap the agent's program to print its input and output. This will let
    you see what the agent is doing in the environment."""
    old_program = agent.program

    def new_program(percept):
        action = old_program(percept)
        print('{} perceives {} and does {}'.format(agent, percept, action))
        return action
    agent.program = new_program
    return agent


def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]"""
    def program(percept):
        #TODO
        # location, status = percept
        # if status == 'Dirty':
        #     return f'Cleaned'
        # elif location == "loc_A":
        #     return f'Right'
        # else:
        #     return f'Left'

        '''fur n*n grid'''
        location, status = percept
        if status == 'Dirty':
            return f'Cleaned'
        return random.choice['Left', 'Right', 'Up', 'Down']

    return Agent(program)


def ModelBasedVacuumAgent():
    """An agent that keeps track of what locations are clean or dirty."""
    model = {loc_A: None, loc_B: None}

    '''fur n*n grid'''
    model_n_n = {}

    def program(percept):
        """Same as ReflexVacuumAgent, except if everything is clean, do NoOp."""
        # location, status = percept
        # model[location] = status
        #
        # if model[loc_A] == 'Clean' and model[loc_B] == 'Clean':
        #     return 'NoOp'
        # else:
        #     model[location] = 'Clean'
        #     if location == loc_A:
        #         return 'Right'
        #     else:
        #         return 'Left'

        '''fur n*n grid'''

        location, status = percept
        model[location] = status

        if status == 'Dirty':
            return 'Suck'

        dirty_pos = [pos for pos, state in model.items() if state == 'Dirty']

        if dirty_pos:
            # zur nahesten dirty position gehen
            target_x, target_y = dirty_pos[0]
            x, y = location
            if target_x > x:
                return 'Right'
            elif target_x < x:
                return 'Left'
            elif target_y > y:
                return 'Up'
            elif target_y < y:
                return 'Down'

        # wenn alles clean ist
        return random.choice(['Up', 'Down', 'Left', 'Right'])

    return Agent(program)

class Environment:
    """Abstract class representing an Environment. 'Real' Environment classes
    inherit from this. Your Environment will typically need to implement:
        percept:           Define the percept that an agent sees.
        execute_action:    Define the effects of executing an action.
                           Also update the agent.performance slot.
    The environment keeps a list of .things and .agents (which is a subset
    of .things). Each agent has a .performance slot, initialized to 0.
    Each thing has a .location slot, even though some environments may not
    need this."""

    def _init_(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Return the percept that the agent sees at this point. (Implement this.)"""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """Change the world to reflect this action. (Implement this.)"""
        raise NotImplementedError

    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        actions = []
        for agent in self.agents:
            actions.append(agent.program(self.percept(agent)))
        for (agent, action) in zip(self.agents, actions):
            self.execute_action(agent, action)
        self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        for step in range(steps):
            self.step()

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        return [thing for thing in self.things
                if thing.location == location and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

    def delete_thing(self, thing):
        """Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)

class VacuumEnvironment(Environment):
    # Einheit mit der sich der Agent in jedwelcher Richtung bewegt = 1

    def _init_(self, max_x=7, max_y=7, dirty_probability=0.4, actions=None):
        super()._init_()
        if actions is None:
            self.actions = ['Left', 'Right', 'Up', 'Down', 'Suck']
        self.max_x = max_x
        self.max_y = max_y
        self.position_und_status = {(x, y): 'Dirty' if random.random() <= dirty_probability else 'Clean' for x in range(self.max_x) for y in range(self.max_y)}

    def percept(self, agent):
        return agent.location, self.position_und_status[agent.location]

    def thing_classes(self):
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def execute_action(self, agent, action):
        x, y = agent.location

        if action == 'Up' and y < self.max_y - 1:
            agent.location = (x, y + 1)
            agent.performance -= 1
        elif action == 'Down' and y > 0:
            agent.location = (x, y - 1)
            agent.performance -= 1
        elif action == 'Right' and x < self.max_x - 1:
            agent.location = (x + 1, y)
            agent.performance -= 1
        elif action == 'Left' and x > 0:
            agent.location = (x - 1, y)
            agent.performance -= 1
        elif action == 'Suck':
            if self.position_und_status[agent.location] == 'Dirty':
                agent.performance += 10
            self.position_und_status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        x = random.randint(0, self.max_x - 1)
        y = random.randint(0, self.max_y - 1)
        return x, y

class TrivialVacuumEnvironment(Environment):

    """This environment has two locations, A and B. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status. This serves as an example of how to implement a simple
    Environment."""

    def _init_(self):
        super()._init_()
        self.status = {loc_A: random.choice(['Clean', 'Dirty']),
                       loc_B: random.choice(['Clean', 'Dirty'])}

    def thing_classes(self):
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        return (agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status; track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        if action == 'Right':
            agent.location = loc_B
            agent.performance -= 1
        elif action == 'Left':
            agent.location = loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B])